<a href="https://colab.research.google.com/github/Pramod-rr/rag_chatbot/blob/main/Simple_rag_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch -q
!pip install transformers -q
!pip install numpy
!pip install langchain -q
!pip install langchain_community -q
!pip install langchain-chroma -q
!pip install sentence_transformers -q

In [2]:
import os
from google.colab import userdata


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
  model_name="Sentence-transformers/all-mpnet-base-v2"
)

/tmp/ipython-input-2429844141.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.2"
huggingface_api_token = userdata.get('HUGGINGFACE_API_TOKEN')
tokenizer = AutoTokenizer.from_pretrained(model_id, token=huggingface_api_token)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype=torch.bfloat16, token=huggingface_api_token)

# Create a text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1, # Use temperature here as before
    trust_remote_code=True,
    device_map="auto",
)

# Initialize the HuggingFace llm using the pipeline
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/tmp/ipython-input-2440115156.py:25: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [5]:
from langchain.schema.output_parser import StrOutputParser

output_parser=StrOutputParser()

In [6]:
!pip install pypdf -qu


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [7]:
!pip install pypdf

In [8]:
from langchain_community.document_loaders import PyPDFLoader

# Load the PDF document
loader = PyPDFLoader("/content/mml-book.pdf")

docs = loader.load()

In [9]:
len(docs)
docs[0]

Document(metadata={'producer': 'pdfTeX-1.40.22', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-02-15T16:21:30+00:00', 'author': 'Marc Peter Deisenroth, A. Aldo Faisal, Cheng Soon Ong', 'title': 'Mathematics for Machine Learning', 'subject': '', 'keywords': '', 'moddate': '2023-02-15T16:21:30+00:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) kpathsea version 6.3.4/dev', 'source': '/content/mml-book.pdf', 'total_pages': 412, 'page': 0, 'page_label': '1'}, page_content='MATHEMATICS  FOR \nMACHINE LEAR NING\nMarc Peter Deisenroth\nA. Aldo Faisal\nCheng Soon On g\nMATHEMATICS FOR MACHINE LEARNING DEISENROTH ET AL.\nThe fundamental mathematical tools needed to understand machine \nlearning include linear algebra, analytic geometry, matrix decompositions, \nvector calculus, optimization, probability and statistics. These topics \nare traditionally taught in disparate courses, making it hard for data \nscience 

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

# Split the documents into chunks
splits = text_splitter.split_documents(docs)

In [15]:
len(splits)

2632

In [16]:
from langchain_chroma import Chroma

# Create a vector store from the document chunks
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
retriever = vectorstore.as_retriever()

In [17]:
from langchain.prompts import ChatPromptTemplate

# Define prompt template
template = """
Answer this question using the provided context only.

{question}

Context:
{context}

Answer:
"""

prompt=ChatPromptTemplate.from_template(template)

In [18]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nAnswer this question using the provided context only.\n\n{question}\n\nContext:\n{context}\n\nAnswer:\n'), additional_kwargs={})])

In [19]:
from langchain.schema.runnable import RunnablePassthrough

chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

response = chain.invoke("")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Human: 
Answer this question using the provided context only.



Context:
[Document(id='d44ecac4-023e-4fd3-a7a5-dc667cef14f9', metadata={'producer': 'pdfTeX-1.40.22', 'total_pages': 412, 'subject': '', 'moddate': '2023-02-15T16:21:30+00:00', 'title': 'Mathematics for Machine Learning', 'page': 10, 'trapped': '/False', 'creationdate': '2023-02-15T16:21:30+00:00', 'source': '/content/mml-book.pdf', 'author': 'Marc Peter Deisenroth, A. Aldo Faisal, Cheng Soon Ong', 'keywords': '', 'creator': 'LaTeX with hyperref', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) kpathsea version 6.3.4/dev', 'page_label': '5'}, page_content='Samuel Ogunmola\nSandeep Mavadia\nSarvesh Nikumbh\nSebastian Raschka\nSenanayak Sesh Kumar Karri\nSeung-Heon Baek\nShahbaz Chaudhary\nShakir Mohamed\nShawn Berry\nSheikh Abdul Raheem Ali\nSheng Xue\nSridhar Thiagarajan\nSyed Nouman Hasany\nSzymon Brych\nThomas B¨uhler\nTimur Sharapov\nTom Melamed\nVincent Adam\nVincent Duto

In [20]:
response = chain.invoke("what is Continuous Optimization")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Human: 
Answer this question using the provided context only.

what is Continuous Optimization

Context:
[Document(id='b54efcd0-0b94-49f4-aff1-bfa64d1d434b', metadata={'title': 'Mathematics for Machine Learning', 'moddate': '2023-02-15T16:21:30+00:00', 'author': 'Marc Peter Deisenroth, A. Aldo Faisal, Cheng Soon Ong', 'creator': 'LaTeX with hyperref', 'producer': 'pdfTeX-1.40.22', 'keywords': '', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) kpathsea version 6.3.4/dev', 'total_pages': 412, 'page_label': '226', 'subject': '', 'source': '/content/mml-book.pdf', 'creationdate': '2023-02-15T16:21:30+00:00', 'trapped': '/False', 'page': 231}, page_content='226 Continuous Optimization\nFigure 7.2 A mind\nmap of the concepts\nrelated to\noptimization, as\npresented in this\nchapter. There are\ntwo main ideas:\ngradient descent\nand convex\noptimization.\nContinuousoptimization\nUnconstrainedoptimization\nConstrainedoptimization\nGradient descen

In [21]:
response = chain.invoke("explain about Geometric vectors")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Human: 
Answer this question using the provided context only.

explain about Geometric vectors

Context:
[Document(id='416609f6-2cce-4a3f-bffb-383fba91d546', metadata={'total_pages': 412, 'creationdate': '2023-02-15T16:21:30+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) kpathsea version 6.3.4/dev', 'trapped': '/False', 'moddate': '2023-02-15T16:21:30+00:00', 'page_label': '17', 'subject': '', 'title': 'Mathematics for Machine Learning', 'author': 'Marc Peter Deisenroth, A. Aldo Faisal, Cheng Soon Ong', 'page': 22, 'producer': 'pdfTeX-1.40.22', 'source': '/content/mml-book.pdf', 'keywords': '', 'creator': 'LaTeX with hyperref'}, page_content='→\nx,\n→\ny can be added, such that\n→\nx+\n→\ny =\n→\nz\nis another geometric vector. Furthermore, multiplication by a scalar\nλ\n→\nx, λ ∈ R, is also a geometric vector. In fact, it is the original vector\nscaled by λ. Therefore, geometric vectors are instances of the vector\nconcepts intro